In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import scipy.io as io

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [2]:
#PATH = 'ml/results_rf_300922_rich_region_ncx_imp.csv' # change to path of interest
PATH = 'ml/results_rf_011022_rich_region_ca1_imp.csv'

In [3]:
results = pd.read_csv(PATH, index_col=0)
complete = results[results.restriction == 'complete']
complete = complete.dropna(how='all', axis=1)
grouped_complete = complete.groupby(by=['restriction', 'modality', 'chunk_size'])
grouped_complete[['auc', 'dev_auc']].quantile(q=[0.25, 0.5, 0.75])

auc   dev_auc
restriction modality      chunk_size                         
complete    morphological 50.0       0.25  0.981237  0.961466
                                     0.50  0.981998  0.997180
                                     0.75  0.986815  1.000000
            spatial       25.0       0.25  0.805020  0.934211
                                     0.50  0.829615  0.966165
                                     0.75  0.858012  0.979323
            temporal      1600.0     0.25  0.765340  0.981673
                                     0.50  0.784229  0.991541
                                     0.75  0.807683  0.994361

In [4]:
wf_ca1 = complete[complete.modality == 'morphological'].dev_auc.to_numpy()
wf_ncx = complete[complete.modality == 'morphological'].auc.to_numpy()

temp_ca1 = complete[complete.modality == 'temporal'].dev_auc.to_numpy()
temp_ncx = complete[complete.modality == 'temporal'].auc.to_numpy()

spat_ca1 = complete[complete.modality == 'spatial'].dev_auc.to_numpy()
spat_ncx = complete[complete.modality == 'spatial'].auc.to_numpy()

statistic, p_val = stats.mannwhitneyu(wf_ca1, wf_ncx)
print(f"Mann-Whitney statistical test results for WF are p-value={p_val} (statistic={statistic})")
print()

statistic, p_val = stats.mannwhitneyu(temp_ca1, temp_ncx)
print(f"Mann-Whitney statistical test results for temporal are p-value={p_val} (statistic={statistic})")
print()

statistic, p_val = stats.mannwhitneyu(spat_ca1, spat_ncx)
print(f"Mann-Whitney statistical test results for spatial are p-value={p_val} (statistic={statistic})")
print()

Mann-Whitney statistical test results for WF are p-value=0.0031361807951584427 (statistic=855.0)

Mann-Whitney statistical test results for temporal are p-value=3.468263926972842e-18 (statistic=0.0)

Mann-Whitney statistical test results for spatial are p-value=4.21486133229539e-18 (statistic=3.0)



In [5]:
cs_50 = complete[complete.chunk_size == 50]
cs_wf = cs_50[cs_50.modality == 'morphological']
wf_aucs = cs_wf.auc.to_numpy()
wf_dev_aucs = cs_wf.dev_auc.to_numpy()

cs_1600 = complete[complete.chunk_size == 1600]
cs_temp = cs_1600[cs_1600.modality == 'temporal']
temp_aucs = cs_temp.auc.to_numpy()
temp_dev_aucs = cs_temp.dev_auc.to_numpy()

cs_25 = complete[complete.chunk_size == 25]
cs_spat = cs_25[cs_25.modality == 'spatial']
spat_aucs = cs_spat.auc.to_numpy()
spat_dev_aucs = cs_spat.dev_auc.to_numpy()

In [6]:
wf_diff = 100 * (wf_dev_aucs - wf_aucs) / wf_dev_aucs
temp_diff = 100 * (temp_dev_aucs - temp_aucs) / temp_dev_aucs
spat_diff = 100 * (spat_dev_aucs - spat_aucs) / spat_dev_aucs

wf_median, wf_prec25, wf_prec75 = np.percentile(wf_diff, [50, 25, 75])
print(f"Median of WF column is {wf_median: .3f} [{wf_prec25: .3f}, {wf_prec75: .3f}]")
temp_median, temp_prec25, temp_prec75 = np.percentile(temp_diff, [50, 25, 75])
print(f"Median of temporal column is {temp_median: .3f} [{temp_prec25: .3f}, {temp_prec75: .3f}]")
spat_median, spat_prec25, spat_prec75 = np.percentile(spat_diff, [50, 25, 75])
print(f"Median of spatial column is {spat_median: .3f} [{spat_prec25: .3f}, {spat_prec75: .3f}]\n")

statistic, p_val = stats.wilcoxon(wf_diff, alternative='greater')
print(f"Wilcoxon statistical test results for wf are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_diff, alternative='greater')
print(f"Wilcoxon statistical test results for temporal are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_diff, alternative='greater')
print(f"Wilcoxon statistical test results for spatial p-value={p_val} (statistic={statistic})\n")

Median of WF column is  0.811 [-1.992,  1.741]
Median of temporal column is  19.386 [ 18.027,  22.318]
Median of spatial column is  13.068 [ 9.420,  16.515]

Wilcoxon statistical test results for wf are p-value=0.5057766851515463 (statistic=636.0)
Wilcoxon statistical test results for temporal are p-value=3.778464727931783e-10 (statistic=1275.0)
Wilcoxon statistical test results for spatial p-value=3.778464727931783e-10 (statistic=1275.0)



In [7]:
statistic, p_val = stats.wilcoxon(wf_aucs - 0.5)
print(f"Mann-Whitney statistical test results for wf inter-region auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_aucs - 0.5)
print(f"Mann-Whitney statistical test results for spatial inter-region auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_aucs - 0.5)
print(f"Mann-Whitney statistical test results for temporal inter-region auc are p-value={p_val} (statistic={statistic})")

statistic, p_val = stats.wilcoxon(wf_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for wf intra-region auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(spat_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for spatial intra-region auc are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.wilcoxon(temp_dev_aucs - 0.5)
print(f"Mann-Whitney statistical test results for temporal intra-region auc are p-value={p_val} (statistic={statistic})")

Mann-Whitney statistical test results for wf inter-region auc are p-value=7.426379562148058e-10 (statistic=0.0)
Mann-Whitney statistical test results for spatial inter-region auc are p-value=7.544976683806673e-10 (statistic=0.0)
Mann-Whitney statistical test results for temporal inter-region auc are p-value=7.556929455863566e-10 (statistic=0.0)
Mann-Whitney statistical test results for wf intra-region auc are p-value=5.25801184027132e-10 (statistic=0.0)
Mann-Whitney statistical test results for spatial intra-region auc are p-value=7.539859267508618e-10 (statistic=0.0)
Mann-Whitney statistical test results for temporal intra-region auc are p-value=7.417972768003161e-10 (statistic=0.0)


## Compare generalizability

In [8]:
PATH_NCX = 'ml/results_rf_300922_rich_region_ncx_imp.csv'
PATH_CA1 = 'ml/results_rf_011022_rich_region_ca1_imp.csv'

In [9]:
results_ncx = pd.read_csv(PATH_NCX, index_col=0).dropna(how='all', axis=1)
results_ca1 = pd.read_csv(PATH_CA1, index_col=0).dropna(how='all', axis=1)

In [10]:
cs_50 = results_ncx[results_ncx.chunk_size == 50]
cs_wf = cs_50[cs_50.modality == 'morphological']
wf_aucs = cs_wf.auc.to_numpy()
wf_dev_aucs = cs_wf.dev_auc.to_numpy()

cs_1600 = results_ncx[results_ncx.chunk_size == 1600]
cs_temp = cs_1600[cs_1600.modality == 'temporal']
temp_aucs = cs_temp.auc.to_numpy()
temp_dev_aucs = cs_temp.dev_auc.to_numpy()

cs_25 = results_ncx[results_ncx.chunk_size == 25]
cs_spat = cs_25[cs_25.modality == 'spatial']
spat_aucs = cs_spat.auc.to_numpy()
spat_dev_aucs = cs_spat.dev_auc.to_numpy()

wf_diff_ncx = 100 * (wf_dev_aucs - wf_aucs) / wf_dev_aucs
temp_diff_ncx = 100 * (temp_dev_aucs - temp_aucs) / temp_dev_aucs
spat_diff_ncx = 100 * (spat_dev_aucs - spat_aucs) / spat_dev_aucs

In [11]:
cs_50 = results_ca1[results_ca1.chunk_size == 50]
cs_wf = cs_50[cs_50.modality == 'morphological']
wf_aucs = cs_wf.auc.to_numpy()
wf_dev_aucs = cs_wf.dev_auc.to_numpy()

cs_1600 = results_ca1[results_ca1.chunk_size == 1600]
cs_temp = cs_1600[cs_1600.modality == 'temporal']
temp_aucs = cs_temp.auc.to_numpy()
temp_dev_aucs = cs_temp.dev_auc.to_numpy()

cs_25 = results_ca1[results_ca1.chunk_size == 25]
cs_spat = cs_25[cs_25.modality == 'spatial']
spat_aucs = cs_spat.auc.to_numpy()
spat_dev_aucs = cs_spat.dev_auc.to_numpy()

wf_diff_ca1 = 100 * (wf_dev_aucs - wf_aucs) / wf_dev_aucs
temp_diff_ca1 = 100 * (temp_dev_aucs - temp_aucs) / temp_dev_aucs
spat_diff_ca1 = 100 * (spat_dev_aucs - spat_aucs) / spat_dev_aucs

In [12]:
statistic, p_val = stats.mannwhitneyu(wf_diff_ca1, wf_diff_ncx)
print(f"Mann-Whitney statistical test results for WF are p-value={p_val} (statistic={statistic})\n")

statistic, p_val = stats.mannwhitneyu(temp_diff_ca1, temp_diff_ncx)
print(f"Mann-Whitney statistical test results for spike-timing are p-value={p_val} (statistic={statistic})\n")

statistic, p_val = stats.mannwhitneyu(spat_diff_ca1, spat_diff_ncx)
print(f"Mann-Whitney statistical test results for spatial are p-value={p_val} (statistic={statistic})")

Mann-Whitney statistical test results for WF are p-value=0.0659846979578251 (statistic=1031.0)

Mann-Whitney statistical test results for spike-timing are p-value=2.2301221981921627e-17 (statistic=31.0)

Mann-Whitney statistical test results for spatial are p-value=2.5250570016990492e-12 (statistic=248.0)
